# Detecção de sarcasmo

O objetivo deste notebook é desenvolver um módulo de detecção automática de sarcasmo em textos em português, mais especificamente notícias. Para isso, foi utilizado uma base de dados composta por notícias de três grandes sites brasileiros, composta por notícias sarcásticas e não sarcásticas. Para a criação do módulo, foram criados dois modelos classificadores que utilizam metodologias diferentes: Uso de algoritmos clássicos de Machine Learning + representação vetorial estática, e Fine-tuning de um modelo transformers multi-língua


## Descrição da estrutura e características do data set

A base de dados foi retirada do repositório [PLNCrawler](https://github.com/schuberty/PLNCrawler), e é estruturada originalmente em três arquivos JSON, que correspondem a cada site de notícias de onde as notícias foram extraídas:
- Sensacionalista: 5006 notícias sarcásticas
- Estadão: 11272 notícias não sarcásticas
- Revista Piauí (seção Herald): 2216 notícias sarcásticas

Cada arquivo possui os seguintes campos para cada notícia:
- is_sarcastic (ou is_sarcasm): booleano, representa o rótulo/label da notícia (sarcástica ou não)
- article_link: string, contem a URL de onde a notícia foi extraída
- headline: string, contem o título da notícia
- text: string, contem o texto da notícia


Carrega as bases de cada site em formato DataFrame:

In [ ]:
import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))
from src.scripts import get_df_sensacionalista
from src.scripts import get_df_estadao
from src.scripts import get_df_the_piaui_herald

# Carrega o arquivo em um DataFrame
df_sensacionalista = get_df_sensacionalista()
df_estadao = get_df_estadao()
df_piaui = get_df_the_piaui_herald()
df_piaui = df_piaui.rename(columns={'is_sarcasm': 'is_sarcastic'}) # Renomeia a coluna para igualizar com os outros DataFrames

display(df_sensacionalista)
display(df_estadao)
display(df_piaui)

Faz a união das três bases em um só DataFrame de forma equilibrada, mantendo 50% de notícias sarcásticas e 50% de não sarcásticas



In [ ]:
# Unir os 3 datasets
from src.scripts import merge_dfs

df = merge_dfs(df_sensacionalista, df_estadao, df_piaui)

num_sarcastic = df['is_sarcastic'].sum()

print(f'Número de amostras sarcásticas: {num_sarcastic}')
print(f'Número de amostras não sarcásticas: {len(df) - num_sarcastic}')

display(df)

# Pré processamento

É importante pontuar que alguns recursos de linguagem que são removidos ou normalizados durante as etapas tradicionais de pré-processamento têm influencia na classificação de ironia em textos.
Por exemplo, sinais de pontuação podem indicar ironia. Por isso, é um parâmetro do pré-processamento remover ou não esse recurso.

Sabendo disso, podem ser passados parâmetros opcionais para a função de pré-processamento que aplicam ou não a transformação.

## Stemming e lemmatization

> "Stemming or lemmatization reduces words to their root form (e.g., "running" becomes "run"), making it easier to analyze language by grouping different forms of the same word." Fonte: https://www.ibm.com/think/topics/natural-language-processing

O processo de stemming e lemmatization são opcionais, mas ambos nunca podem ser aplicados juntos porque eles têm o mesmo propósito com abordagens diferentes.
Dessa forma, se ambos forem ativados só o **lemmatization** será aplicado (por ser mais semântico).

### Fontes para o pré-processamento:

1. [Orientações principais](https://github.com/sharadpatell/Text_preprocessing_steps_for_NLP/blob/main/Text_preprocessing_steps_for_NLP.ipynb) que auxiliaram no passo a passo do pré-processamento.
2. FACELI, K. et al. Inteligência Artificial Uma Abordagem de Aprendizado de Máquina. 2o edição ed.


In [ ]:
from src.preprocessamento import pre_processamento

usar_lemmatization = False
usar_stemming      = False

df = pre_processamento(df, usar_stemming = usar_stemming, usar_lemmatization = usar_lemmatization)

display(df)

## Primeira abordagem para deteccção: Uso de algoritmos clássicos de Machine Learning + representação vetorial estática

Para essa abordagem, o primeiro passo é criar a representação vetorial do texto, pois os computadores não interpretam os textos na linguagem do ser humano. Por isso, é necessário transformá-los para uma representação estruturada que as máquinas consigam processar.

Esse tratamento do texto também faz parte da etapa de feature extraction.
> Feature extraction is the process of converting raw text into numerical representations that machines can analyze and interpret. Fonte: https://www.ibm.com/think/topics/natural-language-processing

Foi escolhido usar a ferramenta Word2Vec para a geração de vetores densos, que capturam o valor semântico das palavras e as relacionam entre sí. É ideal para tarefas de aprendizado de máquina.

In [ ]:
usar_word2vec = False
usar_sequence_transformer = not usar_word2vec
usar_sequence_transformer = False

Aplica Word2Vec na base de dados, gerando a rede neural e retornando os embeddings para cada notícia

In [ ]:
# from scripts.representacao_computacional import aplica_word2vec
# modelo, embeddings = aplica_word2vec(df, nome_coluna='headline')

if (usar_word2vec):
    import subprocess
    import pandas as pd
    import pickle
    
    # Salvar o dataframe temporariamente
    df.to_parquet("temp_input.parquet", engine="pyarrow")  
    
    # Comando para ativar conda env e rodar o script
    subprocess.run([
        "conda", "run", "-n", "word2vec_env", "python",
        "scripts/word2vec_runner.py", "0", "temp_input.parquet", "text", "skip-gram"
    ])
    
    # Recuperar o resultado
    with open("embeddings_output.pkl", "rb") as f:
        embeddings = pickle.load(f)

    with open("indices_validos.pkl", "rb") as f:
        indices_validos = pickle.load(f)
    
    embeddings
    print(embeddings[:5])

###### Treinar um modelo de ML tradicional usando os embeddings Word2Vec

Criados os embeddings, o próximo passo é iniciar o treinamento de um modelo de Machine Learning

Divisão de dados de treino e teste

In [ ]:
if (usar_word2vec):
    import numpy as np
    from sklearn.model_selection import train_test_split
    
    # Convertendo para arrays
    X = np.array(embeddings)
    y = df.iloc[indices_validos]["is_sarcastic"].astype(int).values
    
    print(f"X shape: {X.shape}, y shape: {y.shape}")
    
    # Confirma que estão alinhados
    assert len(X) == len(y)
    print(len(X), len(y))
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )

Testa diversos modelos/algoritmos

In [ ]:
if (usar_word2vec):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.svm import SVC
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.metrics import classification_report, confusion_matrix
    
    modelos = {
        "SVM": SVC(kernel='linear', probability=True),
        "Random Forest": RandomForestClassifier(n_estimators=100),
        "Decision Tree": DecisionTreeClassifier(),
        "Logistic Regression": LogisticRegression(max_iter=1000),
        "KNN": KNeighborsClassifier(n_neighbors=5)
    }
    
    for nome, modelo in modelos.items():
        print(f"\n=== {nome} ===")
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)
        print(classification_report(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))
    

Os resultados foram similares, porém o melhor foi o algoritmo Random Forest, sendo esse o escolhido para o modelo final.

In [ ]:
if (usar_word2vec):
    import joblib

    modelo = RandomForestClassifier(n_estimators=100)
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

    joblib.dump(modelo, "modelo_word2vec.pkl")

### Predição de sarcasmo usando o modelo gerado

In [ ]:
if (usar_word2vec):
    from scripts.preprocessamento import pre_processamento_frase
    import pandas as pd
    import joblib
    import subprocess
    
    # Carrega modelo Word2Vec
    # w2v_model = Word2Vec.load("modelo_word2vec.model")
    
    # Carrega classificador treinado (SVM, Random Forest etc.)
    classificador = joblib.load("modelo_word2vec.pkl")
    
    frase = input("Digite um texto para análise: ")
    
    tokens = pre_processamento_frase(frase)
    
    # salvar os tokens em um arquivo CSV temporário
    pd.DataFrame({"tokens": [tokens]}).to_csv("frase_processada.csv", index=False)
    
    # Roda word2vec na frase processada
    
    subprocess.run([
            "conda", "run", "-n", "word2vec_env", "python",
            "scripts/word2vec_runner.py", "1"
        ], check=True, capture_output=True)
    
    # Carregar os embeddings do arquivo CSV
    vetor = pd.read_csv("vetor_word2vec.csv", header=None).values
    # print(f"[INFO] Vetor carregado do CSV: {vetor_carregado}")
    
    # vetor = vetor_medio(tokens, w2v_model)
    
    
    # Previsão
    pred = classificador.predict(vetor)
    prob = classificador.predict_proba(vetor)[0]
    
    if pred[0] == 1:
        print(f"Sarcamo detectado (confiança: {prob[1]:.2f})")
    else:
        print(f"Sarcamo não detectado (confiança: {prob[0]:.2f})")

## Segunda abordagem para detecção: Fine-tuning de um modelo Sentence Transformer

A segunda abordagem é composta pela escolha de um modelo Transformrers de linguagem, e a partir dele realizer um fine-tuning pra o nosso objetivo.

"Finetuning Sentence Transformer models often heavily improves the performance of the model on your use case, because each task requires a different notion of similarity."
Fonte: https://sbert.net/docs/sentence_transformer/training_overview.html

Antes da aplicação do fine tuning, é importante que o dataset esteja de acordo com a função de perda.
"It is important that your dataset format matches your loss function (or that you choose a loss function that matches your dataset format)"

Para textos curtos (como é o exemplo da headline), o Word2Vec funciona bem. Para textos longos (como é o caso de notícias), pode ser mais efetivo utilizar transformers como BERT.

Encontrar um modelo Sequence Transformer:
- Treinado ou adaptado para pt-BR
- Ser fine-tuning em sentence similarity, feature extraction
- Treinado preferenciamente em notícias
- Usar uma arquitetura encoder compatível com sentence-transformers


Assim foi escolhido o modelo sentence-transformers/xlm-r-bert-base-nli-stsb-mean-tokens

Carrega o modelo original e realiza o ajuste fino

In [ ]:
if (usar_sequence_transformer):
    import subprocess
    import pandas as pd
    import pickle
    from sentence_transformers import SentenceTransformer
    
    print('  Carregando modelo base...')
    # Carrega modelo base
    #modelo = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
    modelo = SentenceTransformer("sentence-transformers/xlm-r-bert-base-nli-stsb-mean-tokens")
    
    # Salva o modelo para ser reutilizado no subprocesso
    modelo.save("modelo_temporario_transformer")
    
    # Salva o DataFrame temporariamente
    print('Salvando o DataFrame temporário...')
    df.to_parquet("temp_input.parquet")
    print('DataFrame temporário salvo.')
    
    # Executa o subprocesso
    print('Iniciando execução do subprocesso...')
    results = subprocess.run([
        "conda", "run", "-n", "transformers_env", "python", "-u",
        "../src/scripts/fine_tuning.py", "temp_input.parquet", "modelo_temporario_transformer"
    ], check=True)
    print('Execução do subprocesso finalizada.')

Carrega o modelo

In [ ]:
carregar_modelo = True

In [ ]:
if carregar_modelo:
    import os
    import joblib
    from sentence_transformers import SentenceTransformer
    
    # Caminhos dos arquivos salvos
    MODELO_DIR = "../modelos/modelo_finetunado_sarcasmo"
    CLASSIFICADOR_PATH = os.path.join(MODELO_DIR, "classificador_logreg.pkl")
    
    def carregar_modelo():
        if not os.path.exists(MODELO_DIR):
            raise FileNotFoundError(f"Diretório '{MODELO_DIR}' não encontrado.")
        if not os.path.exists(CLASSIFICADOR_PATH):
            raise FileNotFoundError(f"Classificador '{CLASSIFICADOR_PATH}' não encontrado.")
    
        print("[INFO] Carregando modelo e classificador...")
        modelo = SentenceTransformer(MODELO_DIR)
        classificador = joblib.load(CLASSIFICADOR_PATH)
        return modelo, classificador
    
    
    modelo, classificador = carregar_modelo()

Predição de sarcasmo usando o modelo gerado

In [ ]:
import os
import joblib
from sentence_transformers import SentenceTransformer
import numpy

def carregar_modelo():
    if not os.path.exists(MODELO_DIR):
        raise FileNotFoundError(f"Diretório '{MODELO_DIR}' não encontrado.")
    if not os.path.exists(CLASSIFICADOR_PATH):
        raise FileNotFoundError(f"Classificador '{CLASSIFICADOR_PATH}' não encontrado.")

    print("[INFO] Carregando modelo e classificador...")
    modelo = SentenceTransformer(MODELO_DIR)
    classificador = joblib.load(CLASSIFICADOR_PATH)
    return modelo, classificador


def prever_sarcasmo(frase, modelo, classificador, limiar=0.5):
    # embedding = modelo.encode([frase], convert_to_tensor=True).cpu().numpy()
    embedding = modelo.encode([frase], convert_to_tensor=True).cpu().tolist()
    prob = classificador.predict_proba(embedding)[0][1]  # Probabilidade de sarcasmo

    if prob >= limiar:
        return "Sarcasmo detectado", prob
    else:
        return "Sarcasmo não detectado", prob


# print("\nDigite uma frase para detectar sarcasmo:")

# frase = input("\n> ")

# if len(frase.strip()) == 0:
#     print("[ERRO] Frase vazia. Tente novamente.")

# resultado, prob = prever_sarcasmo(frase, modelo, classificador)
# print(f"{resultado} (confiança: {prob:.2f})")

# Parte 2: Detecção de Ambiguidade

# Reescrita de frase

In [ ]:
import subprocess

def verificarAmbiguidadePalavra(palavra, contexto):
    result = subprocess.run(
        [
            "conda", "run", "-n", "ambiguidade", "python",
            "../src/ambiguidade.py", contexto, palavra
        ],
        check=True,
        capture_output=True,
        text=True  # Para já retornar string ao invés de bytes
    )

    if (result.stdout.strip() == 'None'):
        return [False, ""]

    return [True, result.stdout.strip()]

def verificarIroniaFrase(frase):
    if len(frase.strip()) == 0:
        print("[ERRO] Frase vazia. Tente novamente.")

    print('Frase: ', frase)
    resultado, prob = prever_sarcasmo(frase, modelo, classificador)
    print('Resultado: ', resultado)

    if resultado.strip() == 'Sarcasmo detectado':
        return True
    return False

In [ ]:
import google.generativeai as genai
API_KEY = ''
genai.configure(api_key = API_KEY)
model = genai.GenerativeModel("gemini-2.5-flash")

from src.reescrita import frases
from src.reescrita import palavras
from src.reescrita import gerarPrompt
#from scripts.reescrita import gerar_texto_com_lmstudio

from src.avaliacao import Avaliacao

# --- Loop Principal do Programa ---

avaliacao = Avaliacao()

while(1):
    print("\n------------------------------------------------------\n")
    print("Digite um texto para análise e reescrita (-1 para finalizar):")
    texto_original = input()

    if texto_original == "-1":
       break

    # 1. Identificação de elementos problemáticos (sarcasmo e ambiguidade)
    palavras_ambiguas_por_frase = {}
    frases_ironicas = []

    lista_frases = frases(texto_original)
    
    for frase in lista_frases:
        if verificarIroniaFrase(frase) == True:
            frases_ironicas.append(frase)
        
        listapalavrasfrase = palavras(frase)
        palavrasAmbiguasNaFrase = []
        for palavrafrase in listapalavrasfrase:
            resultadoambiguidade = verificarAmbiguidadePalavra(palavrafrase, frase)
            if resultadoambiguidade[0] == True:
                palavrasAmbiguasNaFrase.append((palavrafrase, resultadoambiguidade[1]))
        
        if palavrasAmbiguasNaFrase:
            palavras_ambiguas_por_frase[frase] = palavrasAmbiguasNaFrase

    

    print("\n--- Itens Detectados para o Prompt ---")
    print(f"Frases Irônicas: {frases_ironicas}")
    print(f"Palavras Ambíguas por Frase: {palavras_ambiguas_por_frase}")
    print("------------------------------------")

    # 2. Geração do Prompt Otimizado
    prompt_final = gerarPrompt(texto_original, frases_ironicas, palavras_ambiguas_por_frase)
    #print(prompt_final)

    # 3. Geração do Texto Tratado pelo LLM (GEMINI)
    print("\n--- Gerando texto com Gemini (Modelo hard-coded: gemini-2.5-flash) ---")
    texto_reescrito = model.generate_content(prompt_final).text
    #texto_reescrito = gerar_texto_com_lmstudio(prompt_final)

    if texto_reescrito:
        if texto_reescrito.strip().startswith("TEXTO REESCRITO:"):
            texto_reescrito = texto_reescrito.strip()[len("TEXTO REESCRITO:"):].strip()
        
        if texto_reescrito.startswith('"') and texto_reescrito.endswith('"'):
            texto_reescrito = texto_reescrito[1:-1].strip()

        print("\n--- TEXTO REESCRITO ---")
        print(texto_reescrito)
        print("-----------------------")

        print(avaliacao.avaliarReescrita(texto_original, texto_reescrito))
    else:
        print("\nNão foi possível gerar o texto reescrito.")